In [755]:
import pandas as pd
import random
import warnings
from numpy import array 

pd.set_option('display.max_columns', None)




In [756]:
def get_data()->pd.DataFrame:
    df = pd.read_csv('data/data.csv')
    df.drop(columns=['wins', 'losses'], inplace=True)
    df['comp_pct'] = df['pass_cmp']/df['pass_att']
    df = df[df['year'] > 2018]
    df.columns = df.columns.str.replace('_y', 'y')

    return df



def prep_season_data()->pd.DataFrame:
    df = get_data()
    df['ties'] = df['ties'].fillna(0)
    to_not_average = ['year', 'team', 'ties', 'win_loss_perc', 'yds_per_play_offense', 'pass_net_yds_per_att', 'rush_yds_per_att','score_pct', 'turnover_pct','g', "comp_pct", "points_diff", "mov" ]
    for col in df.columns:
        if col not in to_not_average:
            df[col] = df[col]/df['g']
    df['mov'] = df['points_diff']/ df['g']
    return df.drop(columns=['g'])


In [757]:
def get_team_df():
    team_df = pd.read_csv('data/nfl_teams.csv')
    team_df.drop(columns=["team_id_pfr","team_conference_pre2002", "team_division_pre2002" ], inplace=True)
    return team_df

def get_games_df():
    games= pd.read_csv('data/spreadspoke_scores.csv')
    games = games[games['schedule_season'] >2018]
    #getting rid of the first 5 weeks of the season because there will not be enough data to make a prediction
    games = games[games['schedule_week'] != '1']
    games = games[games['schedule_week'] != '2']
    games = games[games['schedule_week'] !='3']
    games = games[games['schedule_week'] !='4']
    games = games[games['schedule_week'] !='5']

    games['spread_favorite_sort'] = abs(games['spread_favorite'])
    
    return games
    

def get_stadiums():
    stadiums = pd.read_csv('data/nfl_stadiums.csv')
    return stadiums

def mege_dfs():
    team_df = get_team_df()
    df = prep_season_data()
    df = df.merge(team_df, left_on='team', right_on='team_name', how='left')
    stadiums = get_stadiums()
    games_df = get_games_df()
    games_df = games_df.merge(stadiums, left_on='stadium', right_on='stadium_name', how='left')
    games_df =games_df[games_df['stadium_neutral'] == False]

    games_df.drop(columns=['stadium_name', 'stadium_location', 'stadium_open', 'stadium_close', 'stadium_type', 'stadium_address', 'stadium_weather_station_zipcode', 'stadium_surface', 'stadium_weather_station', 'stadium_weather_station_name', 'stadium_latitude', 'stadium_longitude', 'stadium_azimuthangle', 'stadium_elevation', 'weather_temperature', 'weather_wind_mph', 'weather_humidity', 'weather_detail', 'stadium_capacity', "stadium"], inplace=True)
    games_df = df.merge(games_df, left_on=['year', 'team'], right_on=['schedule_season', 'team_home' ], how='left')
    games_df = df.merge(games_df, left_on=['year', 'team'], right_on=['schedule_season', 'team_away' ], how='left')
    games_df.columns = games_df.columns.str.replace('_x', '_home')
    games_df.columns = games_df.columns.str.replace('_y', '_away')
    games_df['home_fav'] = games_df['team_favorite_id'] == games_df['team_id_home']
    games_df['home_win'] = games_df['score_home'] > games_df['score_away']
    games_df = games_df.sort_values(by='spread_favorite_sort', ascending=True)
    games_df['over_under_line'] = games_df['over_under_line'].astype(float)
    games_df['over_under_line'] = games_df['over_under_line'].fillna(games_df['over_under_line'].mean())

    columns_to_drop = [
    'spread_favorite_sort', 'stadium_neutral', 'team_favorite_id', 'team_away', 
    'schedule_playoff', 'team_home', 'schedule_season', 'schedule_week', 
    'team_name_away', 'team_name_short_away', 'team_id_away', 
    'team_conference_away', 'team_division_away', 'schedule_date', 
    'team_away', 'team_name_home', 'team_name_short_home', 'team_id_home', 
    'team_conference_home', 'team_division_home', 'year_away', 'year_home', 'ties_home'
    ]
    games_df.drop(columns=columns_to_drop, inplace=True)
    return games_df.head(550)

def bin_data(df):
    df_weather = pd.DataFrame({"Count": df['stadium_weather_type'].value_counts()})
    df_weather['Proportion'] = df_weather['Count'] / df.shape[0]
    #bin moderate and warm together since they make up a smaller portion and warmer weather is not 
    # a bigger factor in october
    df['stadium_weather_type'] = df['stadium_weather_type'].replace(['moderate', 'warm'], 'moderate/warm')
    return df

df= mege_dfs()

df = bin_data(df)
df

,win_loss_perc_home,points_home,points_opp_home,points_diff_home,mov_home,totalyards_home,plays_offense_home,yds_per_play_offense_home,turnovers_home,fumbles_lost_home,first_down_home,pass_cmp_home,pass_att_home,passyds_home,pass_td_home,pass_int_home,pass_netyds_per_att_home,pass_fd_home,rush_att_home,rushyds_home,rush_td_home,rushyds_per_att_home,rush_fd_home,penalties_home,penaltiesyds_home,pen_fd_home,score_pct_home,turnover_pct_home,exp_pts_tot_home,comp_pct_home,win_loss_perc_away,points_away,points_opp_away,points_diff_away,mov_away,totalyards_away,plays_offense_away,yds_per_play_offense_away,turnovers_away,fumbles_lost_away,first_down_away,pass_cmp_away,pass_att_away,passyds_away,pass_td_away,pass_int_away,pass_netyds_per_att_away,pass_fd_away,rush_att_away,rushyds_away,rush_td_away,rushyds_per_att_away,rush_fd_away,penalties_away,penaltiesyds_away,pen_fd_away,score_pct_away,turnover_pct_away,exp_pts_tot_away,ties_away,comp_pct_away,score_home,score_away,spread_favorite,over_under_line,stadium_weather_type,home_fav,home_win
53,0.563,25.125000,20.687500,71,4.437500,362.812500,59.312500,6.1,1.062500,0.562500,19.812500,18.562500,28.000000,223.875000,1.812500,0.500000,0.443750,11.062500,27.812500,138.937500,1.312500,0.312500,6.500000,6.187500,58.250000,2.250000,31.4,8.6,7.242500,0.662946,0.438,22.562500,23.312500,-12,-0.750000,327.375000,63.500000,5.2,1.312500,0.687500,21.250000,19.187500,32.062500,194.250000,1.375000,0.625000,0.356250,10.312500,29.437500,133.125000,1.062500,0.281250,8.187500,4.937500,41.875000,2.750000,36.3,11.3,4.675000,0.0,0.598441,17,31,-1.0,41.5,indoor,True,False
679,0.294,16.882353,21.117647,-72,-4.235294,325.117647,63.411765,5.1,1.411765,0.529412,17.647059,20.294118,33.588235,211.294118,1.058824,0.882353,0.335294,9.764706,26.117647,113.823529,0.647059,0.258824,6.176471,6.647059,57.058824,1.705882,28.6,10.6,-2.448235,0.604203,0.412,20.411765,22.000000,-27,-1.588235,306.235294,57.411765,5.3,1.235294,0.470588,16.529412,15.705882,26.882353,176.235294,0.941176,0.764706,0.358824,8.117647,28.411765,130.000000,0.941176,0.270588,7.000000,6.117647,50.705882,1.411765,35.3,9.6,-0.297647,0.0,0.584245,23,10,-1.0,36.0,moderate/warm,True,True
639,0.412,17.529412,21.117647,-61,-3.588235,296.764706,58.352941,5.1,1.352941,0.705882,16.294118,16.764706,26.823529,171.411765,0.941176,0.647059,0.341176,9.000000,28.647059,125.352941,0.941176,0.258824,6.117647,6.529412,56.235294,1.176471,27.9,10.5,-1.601176,0.625000,0.206,17.000000,24.705882,-131,-7.705882,283.529412,59.705882,4.7,1.647059,0.529412,16.294118,20.705882,34.058824,196.705882,1.176471,1.117647,0.317647,9.352941,23.411765,86.823529,0.411765,0.217647,5.294118,5.176471,37.882353,1.647059,28.8,13.6,-7.093529,1.0,0.607945,10,17,-1.0,39.5,indoor,True,False
87,0.313,21.062500,21.562500,-8,-0.500000,367.437500,62.312500,5.9,1.937500,0.687500,21.812500,24.625000,37.312500,276.625000,1.500000,1.250000,0.437500,13.750000,22.875000,90.812500,0.750000,0.250000,5.625000,6.437500,54.500000,2.437500,38.8,18.1,5.151875,0.659966,0.438,19.562500,26.187500,-106,-6.625000,363.687500,61.812500,5.9,1.062500,0.562500,19.687500,22.937500,32.687500,245.375000,1.375000,0.500000,0.443750,12.125000,27.312500,118.312500,0.812500,0.268750,6.500000,8.000000,71.125000,1.062500,32.7,9.7,4.973125,0.0,0.701721,26,24,-1.0,49.0,moderate/warm,True,True
36,0.375,20.937500,24.562500,-58,-3.625000,340.937500,60.812500,5.6,1.750000,0.437500,19.062500,19.875000,33.687500,222.125000,1.375000,1.312500,0.381250,11.250000,24.562500,118.812500,0.937500,0.300000,5.625000,7.625000,69.125000,2.187500,33.9,14.8,0.846250,0.589981,0.500,18.062500,18.937500,-14,-0.875000,276.750000,58.562500,4.7,1.875000,0.687500,16.562500,19.687500,31.875000,186.312500,1.125000,1.187500,0.343750,9.187500,24.687500,90.437500,0.437500,0.231250,4.687500,6.937500,55.812500,2.687500,28.9,16.0,-5.347500,0.0,0.617647,20,13,-1.0,40.0,cold,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.